# State tables

State machines simplify control flow. One way to represent states and trasitions is with a [state/event table](https://en.wikipedia.org/wiki/Finite-state_machine#State/Event_table).

One way to build them is: the combination of current state (e.g. B) and input (e.g. Y) shows the next state (e.g. C).

|Input \ Current State   | State A  | State B  | State C  |
|:-:|:-:|:-:|:-:|
| Input X  | ...  | ...  | ...  |
| Input Y  | ...  | State C  | ...  |
| Input Z  | ...  | ...  | ...  |

However, the complete action's information is not directly described in the table. It can only be added using *footnotes*. We can achieve this using conditions:

|   State Name  |    Condition(s)   |    Action(s)   |   |
|-------------|-----------------|--------------|:-:|
| Current state |  Entry condition  | Output name(s) |   |
|               |   Exit condition  |     Output     |   |
|               | Virtual condition |     Output     |   |
| Next state    | Condition         | Output         |   |
| Next state    | Condition         | Output         |   |
| ...           | ...               |       ...      |   |
|               |                   |                |   |


<!-- An FSM definition including the full actions information is possible using state tables. -->

Let's implement one. I am going to use pandas instead of raw python lists becuase it has nicer array indexing syntax. 

Before building it, I will state a problem:

We are managing some platform where users can register. When they sign up we ask them for their email and their ID card number. After that they need to validate their address. Some users can be admins. Admins can notify ban users, but those users will go through a revision process before. If the user gets banned they won't be able to register again using that ID card number.

Users can post information to the platform, but only if they have a valid account. Users can deactivate their accounts, but if they are and admin they will not be able to become admin again if they come back.

In [70]:
import numpy as np
import pandas as pd

from enum import Enum, auto, unique
from shortuuid import uuid

from random import choice

Step 1: build the DataFrame

In [40]:
n_states = 5
n_inputs = 10

In [41]:
# I'm going to first build a dictionary and then convert that to a DataFrame.

# state_dict = {
#     f"State {state}": [None for _ in range(n_inputs)] for state in range(n_states)
# }

# [f"Input {i}" for i in range(n_inputs)] + [lambda x: x+1]

In [98]:
@unique
class States(Enum):
    no_account = auto()
    signed_up = auto()
    email_validated = auto()
    admin = auto()
    in_revision = auto()
    deactivated = auto()

In [114]:
@unique
class Inputs(Enum):
    try_sign_up = auto()
    deactivate_account = auto()
    post_information = auto()
    reactivate_account = auto()

We already have a few users in our platform. They are stored in a database (a dictionary for simplicity). For obvious reasons, out database does not have any user with the state `no_account`.

In [100]:
users_db = {uuid(): choice([s for s in States if s.name != "no_account"]) for _ in range(25)}

In [101]:
users_db

{'VRKDpGriG4PreYzNp2rKHq': <States.email_validated: 3>,
 'hz5WxsxBF44a73cHrZAEzP': <States.signed_up: 2>,
 'VLoqfUqkHZchHCKCqeQ45H': <States.email_validated: 3>,
 'Kg7GhzVwNSswJQwDXcWbP5': <States.email_validated: 3>,
 'KixJKMTiTBW8JZSdHYXZYj': <States.in_revision: 5>,
 'atDuYeDtvUHAmfJzaHThFA': <States.email_validated: 3>,
 '5GpqCpMfDKyzJd8MpjNYZC': <States.in_revision: 5>,
 'AvZjAqQ6vBNS2NoRH6GDaV': <States.admin: 4>,
 '8r8BMpxMbmyDNH8x3VNush': <States.in_revision: 5>,
 'oXLsgazxXXj9moz5EmzFS8': <States.admin: 4>,
 'QgajatiW98ubWmFS7Kaumk': <States.email_validated: 3>,
 'J2pyhMe9tj3r3dwyjbu5uq': <States.email_validated: 3>,
 'b8sBDGMUShNidGJpyFXbxb': <States.deactivated: 6>,
 'bUBh4W9X6evHkrP3R4ovK9': <States.in_revision: 5>,
 'MJXnCEJyYVeCnrsMGUyKa5': <States.signed_up: 2>,
 'GUxrgxFnMPci6biJ5TXvok': <States.in_revision: 5>,
 'WF9Q78d9aMaqQqwCZKahdq': <States.email_validated: 3>,
 'cLsjrYpMzTf3r8SF3GtmKY': <States.admin: 4>,
 'LafMMRHcefwHQHcZD6AD87': <States.signed_up: 2>,
 'SdNve2

And now out states table.

In [115]:
table = pd.DataFrame(
    columns=[s.name for s in States],
    index=[i.name for i in Inputs],
)

In [116]:
table

,no_account,signed_up,email_validated,admin,in_revision,deactivated
try_sign_up,NaN,NaN,NaN,NaN,NaN,NaN
deactivate_account,NaN,NaN,NaN,NaN,NaN,NaN
post_information,NaN,NaN,NaN,NaN,NaN,NaN
reactivate_account,NaN,NaN,NaN,NaN,NaN,NaN


We need a way to check the current state. In Python this would normally be done by creating an object which holds the state of its instance. In our case it would me having an instance or `user` that we would need to pass around. By creating `user` as a state machine we could manage it by only giving inputs and letting the state machine manage the processing [link to library]. Here we are doing it in a functional way.

In [104]:
def check_state(user_id: str):
    
    if user_id not in users_db:
        
        return States.no_account
    
    state = users_db[user_id]
    
    return state

In [105]:
check_state("ZRwu4SydyYV2NETiRAnZmM")

<States.no_account: 1>

For each pair input - state there is a callback to do an action. We need to create our stateless functions and the asign each one to a pair:

In [122]:
def sign_up():
    pass

def check_your_email():
    pass

def already_has_account():
    pass

def in_revision():
    pass

def not_allowed():
    pass

def no_account():
    pass

def deactivate():
    pass

def deactivate_admin():
    pass

def already_deactivated():
    pass

def post_info():
    pass


def reactivate():
    pass

In [112]:
table.loc["try_sign_up"]["no_account"] = sign_up
table.loc["try_sign_up"]["signed_up"] = check_your_email
table.loc["try_sign_up"]["email_validated"] = already_has_account
table.loc["try_sign_up"]["admin"] = already_has_account
table.loc["try_sign_up"]["in_revision"] = in_revision
table.loc["try_sign_up"]["deactivated"] = not_allowed

In [119]:
table.loc["deactivate_account"]["no_account"] = no_account
table.loc["deactivate_account"]["signed_up"] = deactivate
table.loc["deactivate_account"]["email_validated"] = deactivate
table.loc["deactivate_account"]["admin"] = deactivate_admin
table.loc["deactivate_account"]["in_revision"] = in_revision
table.loc["deactivate_account"]["deactivated"] = already_deactivated

In [121]:
table.loc["post_information"]["no_account"] = not_allowed
table.loc["post_information"]["signed_up"] = not_allowed
table.loc["post_information"]["email_validated"] = post_info
table.loc["post_information"]["admin"] = post_info
table.loc["post_information"]["in_revision"] = not_allowed
table.loc["post_information"]["deactivated"] = not_allowed

In [123]:
table.loc["reactivate_account"]["no_account"] = no_account
table.loc["reactivate_account"]["signed_up"] = already_has_account
table.loc["reactivate_account"]["email_validated"] = already_has_account
table.loc["reactivate_account"]["admin"] = already_has_account
table.loc["reactivate_account"]["in_revision"] = in_revision
table.loc["reactivate_account"]["deactivated"] = reactivate

In [106]:
def do_input(user_id: str, action: Inputs):
    
    state = check_state(user_id)

In [42]:
def f():
    return 1

In [43]:
states = pd.DataFrame(
    columns=[f"State {s}" for s in range(n_states)],
    index=[f"Input {i}" for i in range(n_inputs)] + [f],
)

In [44]:
def get_states(callback):
    
    return states.loc[lambda x: callback]

How to index a dataframe with a function.

In [45]:
states.loc[lambda x: f]

State 0    NaN
State 1    NaN
State 2    NaN
State 3    NaN
State 4    NaN
Name: <function f at 0x117b2db00>, dtype: object

Define the possible states

In [ ]:
def sign_up():
    
    return states.signed_up

In [ ]:
callbacks = [
    
    
    
]

In [67]:
table.loc["try_sign_up"]["no_account"]

nan

In [58]:
States["no_account"]

<States.no_account: 1>

In [48]:
States.create_account

<States.create_account: 1>

Note on enums from pymotw

Unique Enumeration Values
Enum members with the same value are tracked as alias references to the same member object. Aliases do not cause repeated values to be present in the iterator for the Enum.
```
enum_aliases.py
import enum


class BugStatus(enum.Enum):

    new = 7
    incomplete = 6
    invalid = 5
    wont_fix = 4
    in_progress = 3
    fix_committed = 2
    fix_released = 1

    by_design = 4
    closed = 1


for status in BugStatus:
    print('{:15} = {}'.format(status.name, status.value))

print('\nSame: by_design is wont_fix: ',
      BugStatus.by_design is BugStatus.wont_fix)
print('Same: closed is fix_released: ',
      BugStatus.closed is BugStatus.fix_released)
```

Because by_design and closed are aliases for other members, they do not appear separately in the output when iterating over the Enum. The canonical name for a member is the first name attached to the value.

```
$ python3 enum_aliases.py

new             = 7
incomplete      = 6
invalid         = 5
wont_fix        = 4
in_progress     = 3
fix_committed   = 2
fix_released    = 1

Same: by_design is wont_fix:  True
Same: closed is fix_released:  True
```

In [11]:
states = pd.DataFrame(state_dict)

In [12]:
states

,State 0,State 1,State 2,State 3,State 4
0,None,None,None,None,None
1,None,None,None,None,None
2,None,None,None,None,None
3,None,None,None,None,None
4,None,None,None,None,None
5,None,None,None,None,None
6,None,None,None,None,None
7,None,None,None,None,None
8,None,None,None,None,None
9,None,None,None,None,None


In [5]:
np.zeros((n_inputs, n_states))

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [ ]:
state = 